# *Imports & Setup***

In [17]:
import json
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict


def pretty_print(output):
    print(json.dumps(output, indent=4))


# Memory System

In [18]:
@dataclass
class Memory:
    """Stores conversation history with limited size"""
    messages: List[Dict] = field(default_factory=list)
    max_history: int = 20

    def add(self, role: str, content: str):
        """Add a new message to memory"""
        self.messages.append({
            "role": role,
            "content": content,
            "time": datetime.now().isoformat()
        })
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]

    def get_context(self, last_n: int = 5):
        """Get last `last_n` messages as context"""
        out = ""
        for m in self.messages[-last_n:]:
            out += f"{m['role']}: {m['content']}\n"
        return out


# Intent Classifier Agent


In [19]:
class IntentAgent:
    """Classify user messages into intents and urgency"""
    def classify(self, message: str):
        text = message.lower()
        if "refund" in text:
            return "refund", "high"
        elif "cancel" in text:
            return "cancellation", "high"
        elif "invoice" in text or "bill" in text:
            return "billing", "medium"
        elif "help" in text or "support" in text:
            return "general_help", "low"
        else:
            return "general", "low"


# Reply Generator Agent

In [20]:
class ReplyAgent:
    """Generate replies based on intent and urgency"""
    def create_reply(self, message: str, intent: str, urgency: str):
        responses = {
            "refund": "I understand you want a refund. Please share your order ID so I can assist you further.",
            "cancellation": "I can help you cancel your subscription. Kindly provide your registered email.",
            "billing": "It seems you have a billing concern. Please send your invoice number for verification.",
            "general_help": "Sure, I'm here to help. Could you please share more details?",
            "general": "Thank you for your message. How can I assist you today?"
        }
        return responses.get(intent, responses["general"])


# Escalation Agent

In [21]:
class EscalationAgent:
    """Determine whether the issue needs human escalation"""
    def check(self, intent: str, urgency: str, message: str):
        if urgency == "high":
            return {
                "escalate": True,
                "note": f"Urgent '{intent}' issue. Needs human review."
            }
        else:
            return {
                "escalate": False,
                "note": "No escalation required."
            }


# Coordinator Agent

In [23]:
class Coordinator:
    """Main agent coordinating intent, reply, escalation, and memory"""
    def __init__(self):
        self.intent_agent = IntentAgent()
        self.reply_agent = ReplyAgent()
        self.escalation_agent = EscalationAgent()
        self.memory = Memory()

    def ask(self, message: str):
        # Save user message
        self.memory.add("user", message)

        # Classify intent & urgency
        intent, urgency = self.intent_agent.classify(message)

        # Generate reply
        reply = self.reply_agent.create_reply(message, intent, urgency)

        # Check if escalation needed
        escalation = self.escalation_agent.check(intent, urgency, message)

        # Save agent reply in memory
        self.memory.add("agent", reply)

        # Compile final output
        final_output = {
            "intent": intent,
            "urgency": urgency,
            "reply": reply,
            "escalation": escalation,
            "context": self.memory.get_context()  # last few messages for reference
        }

        return final_output


# Test the Multi-Agent System

In [24]:
coordinator = Coordinator()

# Example conversation
messages = [
    "Hi, I want a refund for my last purchase.",
    "Also, I need help with my invoice.",
    "Can you cancel my subscription?"
]

for msg in messages:
    output = coordinator.ask(msg)
    pretty_print(output)


{
    "intent": "refund",
    "urgency": "high",
    "reply": "I understand you want a refund. Please share your order ID so I can assist you further.",
    "escalation": {
        "escalate": true,
        "note": "Urgent 'refund' issue. Needs human review."
    },
    "context": "user: Hi, I want a refund for my last purchase.\nagent: I understand you want a refund. Please share your order ID so I can assist you further.\n"
}
{
    "intent": "billing",
    "urgency": "medium",
    "reply": "It seems you have a billing concern. Please send your invoice number for verification.",
    "escalation": {
        "escalate": false,
        "note": "No escalation required."
    },
    "context": "user: Hi, I want a refund for my last purchase.\nagent: I understand you want a refund. Please share your order ID so I can assist you further.\nuser: Also, I need help with my invoice.\nagent: It seems you have a billing concern. Please send your invoice number for verification.\n"
}
{
    "intent":

{
    "intent": "refund",
    "urgency": "high",
    "reply": "I understand you want a refund. Please share your order ID so I can assist you further.",
    "escalation": {
        "escalate": true,
        "note": "Urgent 'refund' issue. Needs human review."
    },
    "context": "user: Hi, I want a refund for my last purchase.\nagent: I understand you want a refund. Please share your order ID so I can assist you further.\n"
}
{
    "intent": "billing",
    "urgency": "medium",
    "reply": "It seems you have a billing concern. Please send your invoice number for verification.",
    "escalation": {
        "escalate": false,
        "note": "No escalation required."
    },
    "context": "user: Hi, I want a refund for my last purchase.\nagent: I understand you want a refund. Please share your order ID so I can assist you further.\nuser: Also, I need help with my invoice.\nagent: It seems you have a billing concern. Please send your invoice number for verification.\n"
}
{
    "intent": "cancellation",
    "urgency": "high",
    "reply": "I can help you cancel your subscription. Kindly provide your registered email.",
    "escalation": {
        "escalate": true,
        "note": "Urgent 'cancellation' issue. Needs human review."
    },
    "context": "agent: I understand you want a refund. Please share your order ID so I can assist you further.\nuser: Also, I need help with my invoice.\nagent: It seems you have a billing concern. Please send your invoice number for verification.\nuser: Can you cancel my subscription?\nagent: I can help you cancel your subscription. Kindly provide your registered email.\n"
}